<a href="https://colab.research.google.com/github/openscilabs/isda/blob/main/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ISDA - Independent Structural-Dimensionality Analysis

This notebook demonstrates the ISDA method using the refactored `isda` package.
It reproduces the analysis of synthetic benchmarks, including standard correlation structures and MOP (Multi-Objective Optimization) test cases.

In [ ]:
# Install ISDA 
!pip install --upgrade git+https://github.com/openscilabs/isda.git

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import isda

# Reload for development iteration
import importlib
importlib.reload(isda)

print("ISDA imported successfully.")

## 1. Data Generators & Utilities

### General Helpers

In [ ]:
def _truth(name, intrinsic_dim_expected, blocks_expected, notes=""):
    return {
        "name": name,
        "intrinsic_dim_expected": int(intrinsic_dim_expected),
        "blocks_expected": blocks_expected,
        "notes": notes,
    }

def _mk_block_names(start, size):
    # start is 1-based
    return [f"f{i}" for i in range(start, start + size)]

def _repeat_with_small_noise(base, rng, noise):
    # base: (N,) -> returns perturbed (N,)
    return base + noise * rng.normal(size=base.shape[0])

def _mop_df(Y):
    return pd.DataFrame(Y, columns=[f"f{i+1}" for i in range(Y.shape[1])])

### Validation Utilities
Custom function to evaluate reconstruction fidelity on benchmark datasets.

In [ ]:
def evaluate_reduced_model_fidelity(results_dict):
    """
    Evaluates reconstruction fidelity for a collection of ISDA results.
    Adapted for local notebook usage.
    """
    results_summary = []
    for name, data in results_dict.items():
        Y = data["Y"]
        mis_indices = data.get("best_mis", [])
        truth = data.get("truth", {})

        # Calculate fidelity (F_real)
        if not mis_indices:
            fidelity = 0.0
            ses = 0.0
        else:
            # Use isda library function
            ses_out = isda.calculate_ses(Y, mis_indices, n_perm=1, return_details=True)
            fidelity = ses_out["F_real"]
            ses = ses_out.get("ses", 0.0)

        expected_dim = truth.get("intrinsic_dim_expected", None)
        mis_size = len(mis_indices)

        entry = {
            "Case": name,
            "Selected MIS Size": mis_size,
            "Reconstruction Fidelity (F_real)": fidelity,
            "SES (Structural Evidence Score)": ses
        }
        if expected_dim is not None:
            entry["Expected Intrinsic Dim"] = expected_dim

        results_summary.append(entry)

    df_summary = pd.DataFrame(results_summary)

    # Reorder columns
    cols = ["Case", "Selected MIS Size", "Reconstruction Fidelity (F_real)", "SES (Structural Evidence Score)"]
    if "Expected Intrinsic Dim" in df_summary.columns:
        cols.insert(1, "Expected Intrinsic Dim")
        df_summary = df_summary[cols]

    return df_summary

### Canonical Structure Test Suite (qualitative calibration)

In [ ]:

# --- Battery 1: Standard Correlation Cases ---

def make_case1_independence(N=1000, M=20, seed=123):
    rng = np.random.default_rng(seed)
    Y = rng.normal(size=(N, M))
    cols = [f"f{i+1}" for i in range(M)]
    df = pd.DataFrame(Y, columns=cols)
    truth = _truth(
        name="Case 1 - Total independence",
        intrinsic_dim_expected=M,
        blocks_expected=[[c] for c in cols],
        notes="Each objective is independent (Gaussian noise)."
    )
    return df, truth

def make_case2_total_redundancy(N=1000, M=20, seed=123):
    rng = np.random.default_rng(seed)
    latent = rng.normal(size=(N, 1))
    noise = rng.normal(scale=0.05, size=(N, M))
    Y = latent + noise
    cols = [f"f{i+1}" for i in range(M)]
    df = pd.DataFrame(Y, columns=cols)
    truth = _truth(
        name="Case 2 - Total redundancy",
        intrinsic_dim_expected=1,
        blocks_expected=[cols],
        notes="A single latent; all objectives are noisy copies."
    )
    return df, truth

def make_case3_block_structure(N=1000, M=20, seed=123):
    rng = np.random.default_rng(seed)
    assert M == 20
    latent_blocks = rng.normal(size=(N, 4))
    Y = np.zeros((N, M))
    for b in range(4):
        for j in range(5):
            idx = 5*b + j
            Y[:, idx] = latent_blocks[:, b] + rng.normal(scale=0.2, size=N)
    cols = [f"f{i+1}" for i in range(M)]
    df = pd.DataFrame(Y, columns=cols)
    blocks = [
        [f"f{i}" for i in range(1, 6)],
        [f"f{i}" for i in range(6, 11)],
        [f"f{i}" for i in range(11, 16)],
        [f"f{i}" for i in range(16, 21)],
    ]
    truth = _truth(
        name="Case 3 - Blocks (4 x 5)",
        intrinsic_dim_expected=4,
        blocks_expected=blocks,
        notes="4 independent latents; each generates 5 objectives."
    )
    return df, truth

def make_case4_two_big_blocks(N=1000, M=20, seed=123):
    rng = np.random.default_rng(seed)
    assert M == 20
    latent_blocks = rng.normal(size=(N, 2))
    Y = np.zeros((N, M))
    for i in range(10):
        Y[:, i] = latent_blocks[:, 0] + rng.normal(scale=0.2, size=N)
    for i in range(10, 20):
        Y[:, i] = latent_blocks[:, 1] + rng.normal(scale=0.2, size=N)
    cols = [f"f{i+1}" for i in range(M)]
    df = pd.DataFrame(Y, columns=cols)
    truth = _truth(
        name="Case 4 - Blocks (2 x 10)",
        intrinsic_dim_expected=2,
        blocks_expected=[
            [f"f{i}" for i in range(1, 11)],
            [f"f{i}" for i in range(11, 21)],
        ],
        notes="2 independent latents; each generates 10 objectives."
    )
    return df, truth

def make_case5_chain_structure(N=1000, M=20, seed=123):
    rng = np.random.default_rng(seed)
    Y = np.zeros((N, M))
    Y[:, 0] = rng.normal(size=N)
    for j in range(1, M):
        Y[:, j] = Y[:, j-1] + rng.normal(scale=0.2, size=N)
    cols = [f"f{i+1}" for i in range(M)]
    df = pd.DataFrame(Y, columns=cols)
    truth = _truth(
        name="Case 5 - Chain",
        intrinsic_dim_expected=M,
        blocks_expected=[cols],
        notes="Sequential dependency."
    )
    return df, truth

def make_case6_mixed_structure(N=1000, M=20, seed=123):
    rng = np.random.default_rng(seed)
    assert M == 20
    Y = np.zeros((N, M))
    # First 10: independent
    Y[:, :10] = rng.normal(size=(N, 10))
    # Last 10: two latents
    latent1 = rng.normal(size=N)
    latent2 = rng.normal(size=N)
    for j in range(10, 15):
        Y[:, j] = latent1 + rng.normal(scale=0.2, size=N)
    for j in range(15, 20):
        Y[:, j] = latent2 + rng.normal(scale=0.2, size=N)
    cols = [f"f{i+1}" for i in range(M)]
    df = pd.DataFrame(Y, columns=cols)
    truth = _truth(
        name="Case 6 - Mixed (indep + latents)",
        intrinsic_dim_expected=12,
        blocks_expected=[[f"f{i}"] for i in range(1, 11)] + [[f"f{i}" for i in range(11, 16)], [f"f{i}" for i in range(16, 21)]],
        notes="f1..f10 independent; f11..f15 latent1; f16..f20 latent2."
    )
    return df, truth

def make_case7_pure_conflict_groups(N=1000, M=20, noise=0.05, seed=123, **kwargs):
    rng = np.random.default_rng(seed)
    if M < 2:
        raise ValueError("M must be >= 2")
    M_pos = (M + 1) // 2
    M_neg = M - M_pos
    x = rng.normal(size=N)
    Y_pos = np.column_stack([x + noise * rng.normal(size=N) for _ in range(M_pos)])
    Y_neg = np.column_stack([(-x) + noise * rng.normal(size=N) for _ in range(M_neg)])
    Y = np.column_stack([Y_pos, Y_neg])
    cols = [f"f{i+1}" for i in range(M)]
    Y = pd.DataFrame(Y, columns=cols)
    truth = {
        "name": f"Case 7 - Structural conflict (anti-corr) 2-groups",
        "intrinsic_dim_expected": 2,
        "blocks_expected": [cols[:M_pos], cols[M_pos:]],
        "notes": "Conflict groups (+x and -x).",
    }
    return Y, truth


### Synthetic MOP Test Suite (nferential validation)

In [ ]:
# --- Battery 2: MOP Benchmarks ---

def mopA_monotonic_redundancy(N=1000, seed=123, noise=0.0):
    rng = np.random.default_rng(seed)
    x = rng.uniform(0.0, 1.0, size=N)

    # 20 monotonic transformations (all 1D redundant)
    feats = [
        x,
        2.0 * x + 0.1,
        np.log(1.0 + 9.0 * x),
        x**2,
        np.sqrt(np.maximum(x, 0.0)),
        x**3,
        np.exp(0.5 * x) - 1.0,
        1.0 / (1.0 + np.exp(-10.0 * (x - 0.5))),
        (x + 0.2) ** 2,
        np.log(1.0 + 3.0 * x),
        np.tanh(2.0 * x),
        (1.0 + x) ** 1.5,
        np.clip(x + 0.05, 0, 1),
        np.clip(1.2 * x, 0, 1),
        np.log1p(20.0 * x) / np.log1p(20.0),
        (x + 1e-6) ** 0.25,
        (x + 0.1) ** 3,
        np.sqrt(np.maximum(0.1 + x, 0.0)),
        np.exp(x) - 1.0,
        (x + 0.3) ** 2,
    ]
    Y = np.vstack([_repeat_with_small_noise(f, rng, noise) for f in feats]).T

    truth = _truth(
        name="MOP-A — Monotonic redundancy (1D, M=20)",
        intrinsic_dim_expected=1,
        blocks_expected=[_mk_block_names(1, 20)],
        notes="20 objectives as monotonic (and redundant) transformations of the same latent x."
    )
    return _mop_df(Y), truth

def mopB_tradeoff_with_redundancies(N=1000, seed=123, noise=0.02):
    rng = np.random.default_rng(seed)
    a = rng.uniform(0.0, 1.0, size=N)
    b = rng.uniform(0.0, 1.0, size=N)

    # Plausible latents
    C = 0.6 * a + 0.8 * b            # cost in ~[0,1.4]
    E = b + 0.3 * (1.0 - a)          # consumption in ~[0,1.3]
    P = a * (1.0 - b) + 0.2 * a      # performance can go up to 1.2 -> BUG for Q

    # FIX: force performance to stay in [0,1] so that Q=1-P stays in [0,1]
    P = np.clip(P, 0.0, 1.0)
    Q = 1.0 - P

    # 7 "cost" objectives
    cost_feats = [
        C,
        _repeat_with_small_noise(C, rng, noise),
        1.0 + 2.0 * C,
        np.log1p(9.0 * C),
        np.sqrt(np.maximum(C, 0.0)),
        C**2,
        (C + 0.1) ** 1.5,
    ]

    # 7 "consumption" objectives
    cons_feats = [
        E,
        _repeat_with_small_noise(E, rng, noise),
        np.sqrt(np.maximum(E, 0.0)),
        np.log1p(9.0 * E),
        E**2,
        (E + 0.05),
        (E + 0.2) ** 1.3,
    ]

    # 6 "performance" objectives (minimization via 1-P), with protected domain
    Q_rep = np.clip(_repeat_with_small_noise(Q, rng, noise), 0.0, 1.0)

    perf_feats = [
        Q,
        Q_rep,
        Q**2,
        np.sqrt(np.maximum(Q, 0.0)),
        np.log1p(9.0 * Q),            # now Q ∈ [0,1] -> always valid
        (Q + 0.1) ** 1.2,             # now Q+0.1 ∈ [0.1,1.1] -> always valid
    ]

    feats = cost_feats + cons_feats + perf_feats
    Y = np.vstack(feats).T

    truth = _truth(
        name="MOP-B — Trade-off + redundancies (~2D, M=20)",
        intrinsic_dim_expected=2,
        blocks_expected=[_mk_block_names(1, 7), _mk_block_names(8, 7), _mk_block_names(15, 6)],
        notes="Three families (cost/consumption/performance) with internal redundancies; effective tends to ~2."
    )
    return _mop_df(Y), truth

def mopC_latent_blocks_4x5(N=1000, seed=123, noise=0.02):
    rng = np.random.default_rng(seed)
    u, v, w, z = rng.uniform(0.0, 1.0, size=(4, N))
    eps = rng.normal(size=N)

    b1 = [u, 2*u, u**2, np.sqrt(np.maximum(u,0.0)), np.log1p(9*u)]
    b2 = [v, v+0.5, np.log1p(9*v), v**2, np.sqrt(np.maximum(v,0.0))]
    b3 = [w, w+noise*eps, np.sqrt(np.maximum(w,0.0)), np.log1p(9*w), (w+0.1)**2]
    b4 = [z, (1.0+z)**2, np.exp(z)-1.0, np.log1p(9*z), np.sqrt(np.maximum(z,0.0))]

    feats = b1 + b2 + b3 + b4
    Y = np.vstack(feats).T

    truth = _truth(
        name="MOP-C — Latent blocks (4×5, M=20)",
        intrinsic_dim_expected=4,
        blocks_expected=[_mk_block_names(1,5), _mk_block_names(6,5), _mk_block_names(11,5), _mk_block_names(16,5)],
        notes="Four independent factors; each block (5 objectives) is internally redundant."
    )
    return _mop_df(Y), truth

def mopD_pure_conflict_groups(N=1000, seed=123, noise=0.0):
    rng = np.random.default_rng(seed)
    x = rng.uniform(0.0, 1.0, size=N)

    g1 = [
        x,
        2*x + 0.1,
        np.log1p(9*x),
        x**2,
        np.sqrt(np.maximum(x,0.0)),
        x**3,
        np.tanh(2*x),
        np.log1p(3*x),
        (x+0.2)**2,
        (1.0 + x)**1.5,
    ]
    y = 1.0 - x
    g2 = [
        y,
        2*y + 0.1,
        np.log1p(9*y),
        y**2,
        np.sqrt(np.maximum(y,0.0)),
        y**3,
        np.tanh(2*y),
        np.log1p(3*y),
        (y+0.2)**2,
        (1.0 + y)**1.5,
    ]

    feats = [_repeat_with_small_noise(f, rng, noise) for f in (g1 + g2)]
    Y = np.vstack(feats).T

    truth = _truth(
        name="MOP-D — Structural conflict (anti-corr) 2-groups (M=20)",
        intrinsic_dim_expected=2,
        blocks_expected=[_mk_block_names(1,10), _mk_block_names(11,10)],
        notes="Two internally redundant groups (+x and 1-x), but antagonistic to each other: conflict must be preserved."
    )
    return _mop_df(Y), truth

def mopE_partial_redundancy_noisy(N=1000, seed=123, noise=0.05):
    rng = np.random.default_rng(seed)
    a = rng.uniform(0.0, 1.0, size=N)
    b = rng.uniform(0.0, 1.0, size=N)
    eps = rng.normal(size=N)

    # subfamily A: redundant around 'a' (10)
    A = [
        a,
        a + noise*eps,
        a - noise*eps,
        2*a + 0.1,
        a**2,
        np.sqrt(np.maximum(a,0.0)),
        np.log1p(9*a),
        (a+0.2)**2,
        np.tanh(2*a),
        (1.0+a)**1.2,
    ]

    # subfamily B: "b" (4)
    B = [
        b,
        b + 0.5,
        np.sqrt(np.maximum(b,0.0)),
        np.log1p(9*b),
    ]

    # mixtures/compounds: functions of s=a+b (6)
    s = a + b
    C = [
        s,
        s**2,
        np.sqrt(np.maximum(s,0.0)),
        np.log1p(9*s),
        (s+0.1)**1.5,
        1.0/(1.0+np.exp(-10*(s-1.0))),
    ]

    feats = A + B + C
    Y = np.vstack(feats).T

    truth = _truth(
        name="MOP-E — Partial redundancy + noise (M=20)",
        intrinsic_dim_expected=2,
        blocks_expected=[_mk_block_names(1,10), _mk_block_names(11,4), _mk_block_names(15,6)],
        notes="Trio/quartet of 'a' extended to 10 redundants; 'b' (4); and 6 compounds around s=a+b."
    )
    return _mop_df(Y), truth

def mopF_regime_switching(N=1000, seed=123, sharpness=20.0, noise=0.0):
    rng = np.random.default_rng(seed)
    a = rng.uniform(0.0, 1.0, size=N)
    b = rng.uniform(0.0, 1.0, size=N)

    s = 1.0 / (1.0 + np.exp(-sharpness * (a - 0.5)))
    L = (1.0 - s) * a + s * b

    eps = rng.normal(size=N)

    L_feats = [
        L,
        L**2,
        np.log1p(9*L),
        np.sqrt(np.maximum(L,0.0)),
        (L+0.1)**1.5,
        np.tanh(2*L),
        np.exp(0.5*L)-1.0,
        (L+0.2)**2,
        np.log1p(3*L),
        _repeat_with_small_noise(L, rng, 0.02) if noise == 0.0 else _repeat_with_small_noise(L, rng, noise),
    ]

    b_feats = [
        b,
        np.sqrt(np.maximum(b,0.0)),
        np.log1p(9*b),
        b**2,
        (b+0.1)**1.5,
        np.tanh(2*b),
        np.exp(0.5*b)-1.0,
        (b+0.2)**2,
        np.log1p(3*b),
        _repeat_with_small_noise(b, rng, 0.02) if noise == 0.0 else _repeat_with_small_noise(b, rng, noise),
    ]

    feats = L_feats + b_feats
    Y = np.vstack(feats).T

    truth = _truth(
        name="MOP-F — Regimes (mixture, M=20)",
        intrinsic_dim_expected=2,
        blocks_expected=[_mk_block_names(1,10), _mk_block_names(11,10)],
        notes="10 objectives redundant around L (mixture by regime) + 10 redundant around b; global correlation can be misleading."
    )
    return _mop_df(Y), truth


## 2. Main Execution Loop

In [ ]:
caution = isda.CONSERVATIVE # Set caution level (0 to 1)

def run_cases(cases_list, N=1000):
    all_results = {}
    for name, gen in cases_list:
        print("\n\n" + "=" * 70)
        print(name)
        print("=" * 70)

        Y, truth = gen(N=N)
        print(f"[OK] Data generated: Y.shape = {Y.shape}")
        
        print("\n--- GROUND TRUTH ---")
        print(f"name: {truth['name']}")
        print(f"intrinsic_dim_expected: {truth['intrinsic_dim_expected']}")
        if truth.get("notes"):
            print(f"notes: {truth['notes']}")

        # 1. Regime Diagnosis
        alpha_min, alpha_max, r_max_real, r_max_null = isda.estimate_alpha_interval(Y)
        metrics = isda.diagnose_alpha_regime(alpha_min, alpha_max)
        report_regime = isda.describe_alpha_regime(metrics)
        print(report_regime)
        
        # 2. Decision
        regime = isda.AlphaRegime(metrics["regime"])
        reduction_applied = regime in (isda.AlphaRegime.LIMINAL_SEPARATION, isda.AlphaRegime.IMMEDIATE_SEPARATION)

        if reduction_applied:
            surrogate_final = "MIS_reduced (use highest ranked MIS)"
            alpha_exec = isda.select_alpha(alpha_min, alpha_max, caution)
            print(f"[DECISION] Reduction APPLIED. Final surrogate: {surrogate_final}")
            print(f"[EXEC] alpha_exec (via select_alpha with caution={caution}) = {alpha_exec:.6g}")
        else:
            surrogate_final = "MIS_complete (all objectives; no reduction)"
            alpha_exec = max(alpha_min, alpha_max)
            print(f"[DECISION] Reduction NOT applied. Final surrogate: {surrogate_final}")
            print(f"[EXEC] alpha_exec = {alpha_exec:.6g} (for inspection only)")

        # 3. Execution
        res = isda.isda_significance(Y, alpha=alpha_exec)
        
        # Report correlations
        print(res['corr_report'])
        
        print(f"\nCase: {name}")
        print(f"N (samples) = {res['N']}, M (objectives) = {res['M']}")
        print(f"Significance level (alpha) = {res['alpha']}")
        print(f"\nNumber of clusters = {len(res['components'])}")
        
        num_mis = len(res['mis_sets'])
        print(f"Number of MIS = {num_mis}")
        
        print("\nMIS Ranking (top 3 rank 1):")
        if 1 in res['rank_groups']:
            for m in res['rank_groups'][1][:3]:
                print(f"  MIS rank 1: {m['mis_labels']}")
                print(f"    metrics: size={m['size']}, neigh={m['neighborhood']}, span={m['span']}")
        if len(res['rank_groups'][1]) > 3:
            print(f"    ... (+ {len(res['rank_groups'][1])-3} others in rank 1)")

        # 4. Validation (SES - Structural Evidence Score)
        best_mis = []
        ses_out = {}
        if res["mis_ranked"]:
            best_mis = res["mis_ranked"][0]["mis_indices"]
            ses_out = isda.calculate_ses(Y, best_mis)
            print(isda.explain_ses(ses_out, name=name))
        
        # 5. Visualisation
        viz = isda.plot_custom_isda_graph(
            res,
            title=f"{name} — alpha={alpha_exec:.3g} — regime={regime.name}",
            show_removed=False
        )
        plt.show()
        plt.close(viz['fig'])
        
        all_results[name] = {
            "truth": truth,
            "Y": Y,
            "regime": regime,
            "res_isda": res,
            "best_mis": best_mis,
            "ses_results": ses_out
        }
    return all_results

battery1 = [
    ("Case 1 - Total independence", make_case1_independence),
    ("Case 2 - Total redundancy", make_case2_total_redundancy),
    ("Case 3 - Blocks (4 x 5)", make_case3_block_structure),
    ("Case 4 - Blocks (2 x 10)", make_case4_two_big_blocks),
    ("Case 5 - Chain", make_case5_chain_structure),
    ("Case 6 - Mixed (indep + latents)", make_case6_mixed_structure),
    ("Case 7 - Structural conflict (anti-corr) with groups", make_case7_pure_conflict_groups),
]

battery2 = [
    ("MOP-A — Monotonic redundancy", mopA_monotonic_redundancy),
    ("MOP-B — Trade-off", mopB_tradeoff_with_redundancies),
    ("MOP-C — Latent blocks (4×5)", mopC_latent_blocks_4x5),
    ("MOP-D — Structural conflict", mopD_pure_conflict_groups),
    ("MOP-E — Partial redundancy", mopE_partial_redundancy_noisy),
    ("MOP-F — Regimes", mopF_regime_switching),
]

print("\n=== RUNNING STANDARD CORRELATION BATTERY ===")
_ = run_cases(battery1)

print("\n\n=== RUNNING MOP BENCHMARK BATTERY ===")
mop_results = run_cases(battery2)

mop_fidelity_df = evaluate_reduced_model_fidelity(mop_results)
print("\n--- ISDA Reconstruction Fidelity Evaluation for MOP Cases ---")
print(mop_fidelity_df.to_markdown(index=False))
